In [63]:
import rasterio 
from shapely.geometry import box, Point
from shapely import get_coordinates
from rasterio.mask import  geometry_mask
import geopandas as gpd
import glob, os

In [69]:
image_dir = 'sample_datasets/image'
shape_path = 'sample_datasets/annotation.geojson'
mask_out_path = 'sample_datasets/mask'

gdf = gpd.read_file(shape_path)
gdf = gpd.GeoSeries(gdf.geometry.apply(lambda geom : [Point(xy) for xy in get_coordinates(geom)[:-1]]).explode()).reset_index(drop=True)

for image_path in glob.glob(os.path.join(image_dir, '*.tif')):

    filename = os.path.basename(image_path)
    with rasterio.open(image_path, 'r') as src:
        bbox = box(*src.bounds)
        image = src.read()
        points = gdf[bbox.contains(gdf)].values
        if len(points) == 0:
            print(image_path, '<= image has no annotation !')
            continue
        
        mask = geometry_mask(points, out_shape=image[0,:,:].shape, transform=src.transform, invert=True) * 255
        out_profile = {
            **src.profile,
            'count' : 1,
        }
        with rasterio.open(os.path.join(mask_out_path, filename), 'w', **out_profile) as dst:
            dst.write(mask, 1)
            
    print('completed =>', image_path)

./image\1023_17.tif <= image has no annotation !
./image\1023_20.tif <= image has no annotation !
./image\1023_21.tif <= image has no annotation !
./image\1023_24.tif <= image has no annotation !
./image\1023_28.tif <= image has no annotation !
./image\1023_29.tif <= image has no annotation !
./image\1023_34.tif <= image has no annotation !
./image\1023_36.tif <= image has no annotation !
./image\1023_37.tif <= image has no annotation !
./image\1023_4.tif <= image has no annotation !
./image\1023_40.tif <= image has no annotation !
./image\1023_42.tif <= image has no annotation !
./image\1023_45.tif <= image has no annotation !
./image\1023_49.tif <= image has no annotation !
./image\1023_50.tif <= image has no annotation !
./image\1023_60.tif <= image has no annotation !
./image\1023_71.tif <= image has no annotation !
completed => ./image\1023_78.tif
./image\1023_8.tif <= image has no annotation !
./image\1023_9.tif <= image has no annotation !
